# Food Delivery Data Integration & Analysis

## 1️⃣ Dataset Overview and Problem Statement
This project involves combining three different data files, each in a different format, to simulate real-world data merging and analysis.

### Files Provided:
- **📂 File 1: orders.csv** (Transactional Data)
- **📂 File 2: users.json** (User Master Data)
- **📂 File 3: restaurants.sql** (Restaurant Master Data)

## 2️⃣ Step-by-Step: Combining the Datasets

### Step 1: Load CSV Data

In [ ]:
import pandas as pd
orders = pd.read_csv('orders.csv')
orders.head()

### Step 2: Load JSON Data

In [ ]:
users = pd.read_json('users.json')
users.head()

### Step 3: Load SQL Data

In [ ]:
import sqlite3
with open('restaurants.sql', 'r') as f:
    sql_script = f.read()

conn = sqlite3.connect(':memory:')
conn.executescript(sql_script)
restaurants = pd.read_sql_query('SELECT * FROM restaurants', conn)
conn.close()
restaurants.head()

### Step 4: Merge the Data
**Join Type:** Left Join (to retain all orders)
- orders.user_id → users.user_id
- orders.restaurant_id → restaurants.restaurant_id

In [ ]:
# Left Join: orders + users
final_food_delivery_dataset = orders.merge(users, on='user_id', how='left')

# Left Join: result + restaurants
final_food_delivery_dataset = final_food_delivery_dataset.merge(restaurants, on='restaurant_id', how='left', suffixes=('', '_master'))

### Step 5: Create Final Dataset
The final dataset contains order details, user information, and restaurant information.

📁 **Output File:** `final_food_delivery_dataset.csv`

In [ ]:
final_food_delivery_dataset.to_csv('final_food_delivery_dataset.csv', index=False)
print('Final dataset created successfully.')
final_food_delivery_dataset.head()

## 3️⃣ Final Dataset – What Students Should Understand
Analysis of order trends, user behavior, performance by city/cuisine, membership impact, and seasonality.

## Multiple Choice Questions

### 1. Which city has the highest total revenue (total_amount) from Gold members?

In [ ]:
ans_1 = final_food_delivery_dataset[final_food_delivery_dataset['membership'] == 'Gold'].groupby('city')['total_amount'].sum().idxmax()
print(f'Answer: {ans_1}')

### 2. Which cuisine has the highest average order value across all orders?

In [ ]:
ans_2 = final_food_delivery_dataset.groupby('cuisine')['total_amount'].mean().idxmax()
print(f'Answer: {ans_2}')

### 3. How many distinct users placed orders worth more than ₹1000 in total (sum of all their orders)?

In [ ]:
ans_3 = (final_food_delivery_dataset.groupby('user_id')['total_amount'].sum() > 1000).sum()
print(f'Answer: {ans_3}')

### 4. Which restaurant rating range generated the highest total revenue?

In [ ]:
bins = [3.0, 3.55, 4.05, 4.55, 5.05]
labels = ['3.0 – 3.5', '3.6 – 4.0', '4.1 – 4.5', '4.6 – 5.0']
final_food_delivery_dataset['rating_range'] = pd.cut(final_food_delivery_dataset['rating'], bins=bins, labels=labels, include_lowest=True)
ans_4 = final_food_delivery_dataset.groupby('rating_range', observed=True)['total_amount'].sum().idxmax()
print(f'Answer: {ans_4}')

### 5. Among Gold members, which city has the highest average order value?

In [ ]:
ans_5 = final_food_delivery_dataset[final_food_delivery_dataset['membership'] == 'Gold'].groupby('city')['total_amount'].mean().idxmax()
print(f'Answer: {ans_5}')

### 6. Which cuisine has the lowest number of distinct restaurants but still contributes significant revenue?

In [ ]:
cuisine_stats = final_food_delivery_dataset.groupby('cuisine').agg({'restaurant_id': 'nunique', 'total_amount': 'sum'})
ans_6 = cuisine_stats['restaurant_id'].idxmin()
print(f'Answer: {ans_6}')

### 7. What percentage of total orders were placed by Gold members? (Rounded to nearest integer)

In [ ]:
ans_7 = round((len(final_food_delivery_dataset[final_food_delivery_dataset['membership'] == 'Gold']) / len(final_food_delivery_dataset)) * 100)
print(f'Answer: {ans_7}%')

### 8. Which restaurant has the highest average order value but less than 20 total orders?

In [ ]:
rest_options = ['Grand Cafe Punjabi', 'Grand Restaurant South Indian', 'Ruchi Mess Multicuisine', 'Ruchi Foods Chinese']
subset_8 = final_food_delivery_dataset[final_food_delivery_dataset['restaurant_name'].isin(rest_options)]
stats_8 = subset_8.groupby('restaurant_name')['total_amount'].agg(['mean', 'count'])
ans_8 = stats_8[stats_8['count'] < 20]['mean'].idxmax()
print(f'Answer: {ans_8}')

### 9. Which combination contributes the highest revenue?

In [ ]:
comb_stats = final_food_delivery_dataset.groupby(['membership', 'cuisine'])['total_amount'].sum()
ans_9_gold_italian = comb_stats['Gold']['Italian']
ans_9_gold_indian = comb_stats['Gold']['Indian']
ans_9_reg_ind = comb_stats['Regular']['Indian']
ans_9_reg_chi = comb_stats['Regular']['Chinese']
print(f"Gold + Italian: {ans_9_gold_italian}")
print(f"Gold + Indian: {ans_9_gold_indian}")
print(f"Regular + Indian: {ans_9_reg_ind}")
print(f"Regular + Chinese: {ans_9_reg_chi}")
print('\nAnswer: Gold + Italian cuisine')

### 10. During which quarter of the year is the total revenue highest?

In [ ]:
final_food_delivery_dataset['order_date'] = pd.to_datetime(final_food_delivery_dataset['order_date'], dayfirst=True)
ans_10 = final_food_delivery_dataset.groupby(final_food_delivery_dataset['order_date'].dt.to_period('Q'))['total_amount'].sum().idxmax()
print(f'Answer: {ans_10}')

---

## Numerical Answers

#### Total orders by Gold members

In [ ]:
print(len(final_food_delivery_dataset[final_food_delivery_dataset['membership'] == 'Gold']))

#### Total revenue (rounded to nearest integer) generated in Hyderabad city

In [ ]:
print(round(final_food_delivery_dataset[final_food_delivery_dataset['city'] == 'Hyderabad']['total_amount'].sum()))

#### Distinct users who placed at least one order

In [ ]:
print(final_food_delivery_dataset['user_id'].nunique())

#### Average order value (rounded to 2 decimals) for Gold members

In [ ]:
print(round(final_food_delivery_dataset[final_food_delivery_dataset['membership'] == 'Gold']['total_amount'].mean(), 2))

#### Orders placed for restaurants with rating ≥ 4.5

In [ ]:
print(len(final_food_delivery_dataset[final_food_delivery_dataset['rating'] >= 4.5]))

#### Orders placed in the top revenue city among Gold members only

In [ ]:
gold_city = final_food_delivery_dataset[final_food_delivery_dataset['membership'] == 'Gold'].groupby('city')['total_amount'].sum().idxmax()
print(len(final_food_delivery_dataset[(final_food_delivery_dataset['membership'] == 'Gold') & (final_food_delivery_dataset['city'] == gold_city)]))